In [1]:
import os, socket, threading, webbrowser
from flask import Flask, request, render_template
from utils.extract import extract_jd_text, extract_resumes
from utils.scorer import compute_scores

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        jd_text = extract_jd_text(request)
        if not jd_text:
            print("❌ No JD text provided.")
            return render_template('index.html', error="Please upload or paste a JD.")

        mandatory_skills = [s.strip() for s in request.form['mandatory_skills'].split(',') if s.strip()]
        must_have_skills = [s.strip() for s in request.form['must_have_skills'].split(',') if s.strip()]

        resumes = extract_resumes('resumes')[:10]  # limit to 10 resumes
        print(f"📥 Found {len(resumes)} valid resumes.")

        if not resumes:
            return render_template('index.html', error="No valid resumes found.")

        results = compute_scores(jd_text, resumes, mandatory_skills, must_have_skills)

        print(f"✅ Successfully scored {len(results)} resumes.\n")
        return render_template('results.html', results=results)

    return render_template('index.html')

def run_app():
    sock = socket.socket()
    sock.bind(('', 0))
    _, port = sock.getsockname()
    sock.close()

    url = f'http://localhost:{port}'
    threading.Thread(target=lambda: app.run(port=port, debug=False, use_reloader=False)).start()
    print(f"\n🚀 Opening in browser: {url}")
    webbrowser.open(url)

if __name__ == '__main__':
    run_app()



🚀 Opening in browser: http://localhost:59772
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:59772
Press CTRL+C to quit
127.0.0.1 - - [02/Jun/2025 16:22:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:22:11] "GET /static/ey_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2025 16:22:11] "GET /favicon.ico HTTP/1.1" 404 -


In [1]:
#new

In [2]:
import os
import socket
import threading
import webbrowser
from flask import Flask, request, render_template, send_from_directory
from utils.extract import extract_jd_text, extract_resumes
from utils.scorer import compute_scores
from utils.excel_report import export_to_excel

app = Flask(__name__)

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        jd_text = extract_jd_text(request)
        if not jd_text:
            return render_template('index.html', error="Please upload or paste a JD.", results=[], excel_filename=None)

        mandatory_skills = [s.strip() for s in request.form['mandatory_skills'].split(',') if s.strip()]
        must_have_skills = [s.strip() for s in request.form['must_have_skills'].split(',') if s.strip()]

        resumes = extract_resumes('resumes')[:10]  # Max 10 resumes
        if not resumes:
            return render_template('index.html', error="No valid resumes found.", results=[], excel_filename=None)

        results = compute_scores(jd_text, resumes, mandatory_skills, must_have_skills)
        excel_filename = export_to_excel(results)

        return render_template('results.html', results=results, excel_filename=excel_filename)

    # For GET request
    return render_template('index.html', results=[], excel_filename=None)

@app.route('/outputs/<filename>')
def download_excel(filename):
    return send_from_directory('outputs', filename, as_attachment=True)

def run_app():
    sock = socket.socket()
    sock.bind(('', 0))
    _, port = sock.getsockname()
    sock.close()

    url = f'http://localhost:{port}'
    threading.Thread(target=lambda: app.run(port=port, debug=False, use_reloader=False)).start()
    print(f"\n🚀 Opening in browser: {url}")
    webbrowser.open(url)

if __name__ == '__main__':
    run_app()



🚀 Opening in browser: http://localhost:51938
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:51938
Press CTRL+C to quit
127.0.0.1 - - [19/Jun/2025 11:44:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2025 11:44:22] "GET /static/ey_logo.png HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2025 11:44:22] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [19/Jun/2025 11:45:08] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2025 11:45:08] "GET /static/ey_logo.png HTTP/1.1" 304 -


🔍 Scoring resume 1: Pathan Nagul Meeravali_ Revenue Assurance.pdf
🔍 Scoring resume 2: Priti Mallick_Revenue Assurance .pdf
🔍 Scoring resume 3: RochellePeres_Analyst - Revenue Assurance.pdf
🔍 Scoring resume 4: Vinit Tandel_ Revenue Assurance.pdf
🔍 Scoring resume 5: Vishal_Kulkarni_Revenue Assurance.pdf
✅ Excel report saved at: outputs\Resume_Score_Report_2025-06-19 11-45-08.xlsx


127.0.0.1 - - [19/Jun/2025 11:45:32] "GET /outputs/Resume_Score_Report_2025-06-19%2011-45-08.xlsx HTTP/1.1" 200 -
127.0.0.1 - - [19/Jun/2025 11:58:14] "GET / HTTP/1.1" 200 -
